# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 10:00AM,240945,10,CTF0011295,"Citieffe, Inc.",Released
1,Jul 1 2022 9:57AM,240944,10,CTF0011294,"Citieffe, Inc.",Released
2,Jul 1 2022 9:51AM,240943,10,8217524,"Citieffe, Inc.",Released
3,Jul 1 2022 9:36AM,240940,12,HH-35227,Hush Hush,Released
4,Jul 1 2022 9:36AM,240940,12,HH-35564,Hush Hush,Released
5,Jul 1 2022 9:36AM,240940,12,HH-35565,Hush Hush,Released
6,Jul 1 2022 9:36AM,240940,12,HH-35566,Hush Hush,Released
7,Jul 1 2022 9:36AM,240940,12,HH-35567,Hush Hush,Released
8,Jul 1 2022 9:36AM,240940,12,HH-35568,Hush Hush,Released
9,Jul 1 2022 9:36AM,240940,12,HH-35569,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,240940,Released,37
43,240941,Released,1
44,240943,Released,1
45,240944,Released,1
46,240945,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240940,NaN,NaN,37.0
240941,NaN,NaN,1.0
240943,NaN,NaN,1.0
240944,NaN,NaN,1.0
240945,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240847,28.0,1.0,0.0
240873,0.0,0.0,1.0
240878,0.0,0.0,1.0
240892,0.0,1.0,0.0
240893,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240847,28,1,0
240873,0,0,1
240878,0,0,1
240892,0,1,0
240893,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240847,28,1,0
1,240873,0,0,1
2,240878,0,0,1
3,240892,0,1,0
4,240893,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240847,28,1,
1,240873,,,1
2,240878,,,1
3,240892,,1,
4,240893,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 10:00AM,240945,10,"Citieffe, Inc."
1,Jul 1 2022 9:57AM,240944,10,"Citieffe, Inc."
2,Jul 1 2022 9:51AM,240943,10,"Citieffe, Inc."
3,Jul 1 2022 9:36AM,240940,12,Hush Hush
40,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC"
41,Jul 1 2022 9:17AM,240939,10,ISDIN Corporation
42,Jul 1 2022 9:17AM,240938,10,ISDIN Corporation
56,Jul 1 2022 9:14AM,240925,15,"Virtus Pharmaceuticals, LLC"
62,Jul 1 2022 9:14AM,240937,12,"NBTY Global, Inc."
64,Jul 1 2022 9:13AM,240924,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 1 2022 10:00AM,240945,10,"Citieffe, Inc.",,,1
1,Jul 1 2022 9:57AM,240944,10,"Citieffe, Inc.",,,1
2,Jul 1 2022 9:51AM,240943,10,"Citieffe, Inc.",,,1
3,Jul 1 2022 9:36AM,240940,12,Hush Hush,,,37
4,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",,,1
5,Jul 1 2022 9:17AM,240939,10,ISDIN Corporation,,,1
6,Jul 1 2022 9:17AM,240938,10,ISDIN Corporation,,,14
7,Jul 1 2022 9:14AM,240925,15,"Virtus Pharmaceuticals, LLC",,,6
8,Jul 1 2022 9:14AM,240937,12,"NBTY Global, Inc.",,1,1
9,Jul 1 2022 9:13AM,240924,15,"Person & Covey, Inc.",,10,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 10:00AM,240945,10,"Citieffe, Inc.",1,,
1,Jul 1 2022 9:57AM,240944,10,"Citieffe, Inc.",1,,
2,Jul 1 2022 9:51AM,240943,10,"Citieffe, Inc.",1,,
3,Jul 1 2022 9:36AM,240940,12,Hush Hush,37,,
4,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",1,,
5,Jul 1 2022 9:17AM,240939,10,ISDIN Corporation,1,,
6,Jul 1 2022 9:17AM,240938,10,ISDIN Corporation,14,,
7,Jul 1 2022 9:14AM,240925,15,"Virtus Pharmaceuticals, LLC",6,,
8,Jul 1 2022 9:14AM,240937,12,"NBTY Global, Inc.",1,1,
9,Jul 1 2022 9:13AM,240924,15,"Person & Covey, Inc.",2,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 10:00AM,240945,10,"Citieffe, Inc.",1,,
1,Jul 1 2022 9:57AM,240944,10,"Citieffe, Inc.",1,,
2,Jul 1 2022 9:51AM,240943,10,"Citieffe, Inc.",1,,
3,Jul 1 2022 9:36AM,240940,12,Hush Hush,37,,
4,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 10:00AM,240945,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Jul 1 2022 9:57AM,240944,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jul 1 2022 9:51AM,240943,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 1 2022 9:36AM,240940,12,Hush Hush,37.0,NaN,NaN
4,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 10:00AM,240945,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Jul 1 2022 9:57AM,240944,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jul 1 2022 9:51AM,240943,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 1 2022 9:36AM,240940,12,Hush Hush,37.0,0.0,0.0
4,Jul 1 2022 9:27AM,240941,10,"Snap Medical Industries, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4095693,122.0,0.0,0.0
12,481877,38.0,1.0,0.0
15,1204541,32.0,14.0,28.0
16,1204512,14.0,1.0,0.0
19,1204525,6.0,19.0,3.0
20,481827,6.0,6.0,0.0
22,722734,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4095693,122.0,0.0,0.0
1,12,481877,38.0,1.0,0.0
2,15,1204541,32.0,14.0,28.0
3,16,1204512,14.0,1.0,0.0
4,19,1204525,6.0,19.0,3.0
5,20,481827,6.0,6.0,0.0
6,22,722734,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,122.0,0.0,0.0
1,12,38.0,1.0,0.0
2,15,32.0,14.0,28.0
3,16,14.0,1.0,0.0
4,19,6.0,19.0,3.0
5,20,6.0,6.0,0.0
6,22,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,122.0
1,12,Released,38.0
2,15,Released,32.0
3,16,Released,14.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Completed,0.0,0.0,28.0,0.0,3.0,0.0,0.0
Executing,0.0,1.0,14.0,1.0,19.0,6.0,2.0
Released,122.0,38.0,32.0,14.0,6.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Completed,0.0,0.0,28.0,0.0,3.0,0.0,0.0
1,Executing,0.0,1.0,14.0,1.0,19.0,6.0,2.0
2,Released,122.0,38.0,32.0,14.0,6.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Completed,0.0,0.0,28.0,0.0,3.0,0.0,0.0
1,Executing,0.0,1.0,14.0,1.0,19.0,6.0,2.0
2,Released,122.0,38.0,32.0,14.0,6.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()